In [1]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
data = open('min-char-rnn-tensorflow.py', 'r').read()  # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

char2idx = { ch:i for i, ch in enumerate(chars)}
idx2char = np.array(chars)

In [3]:
def get_raw_data():
    data_as_int = np.array(list(map(char2idx.get,data)))
    return data_as_int[0:-1],data_as_int[1:]


In [4]:
get_raw_data()

state_size = 100
batch_size = 5
seq_length = 25
learning_rate = 1e-1

def get_batch_seq(data):
    raw_x, raw_y = data
    batch_partition_length = len(raw_x) // batch_size
#     print(batch_partition_length)
#     print(raw_x[:-(len(raw_x)%batch_partition_length)])
    data_x=raw_x[:-(len(raw_x)%batch_partition_length)].reshape(-1,batch_partition_length)
    data_y=raw_y[:-(len(raw_x)%batch_partition_length)].reshape(-1,batch_partition_length)
    
    epoch_steps = batch_partition_length // seq_length
    for step in range(epoch_steps):        
        x = data_x[:, step*seq_length:(step+1)*seq_length]
        y = data_y[:, step*seq_length:(step+1)*seq_length]
        yield x,y           

def get_epoch(n):
    for i in range(n):
        yield get_batch_seq(get_raw_data())


In [5]:
class CharRnnModel():
    
    def __init__(self):
        self.sess = tf.Session()
        
    def create_compute_graph(self):

        with tf.variable_scope(str(id(self)) + 'rnn_cell'):
            w = tf.get_variable('w',[vocab_size + state_size, state_size])
            b = tf.get_variable('b',[state_size])

        def rnn_cell(rnn_input,pre_state):

            with tf.variable_scope(str(id(self)) + 'rnn_cell',reuse=True):
                w = tf.get_variable('w',[vocab_size + state_size, state_size])
                b = tf.get_variable('b',[state_size])
            return tf.tanh(tf.matmul(tf.concat([rnn_input,pre_state],axis=1), w) + b)
        
        # def create_compute_graph():
        x = tf.placeholder(tf.int32, [None, seq_length])
        y = tf.placeholder(tf.int32, [None, seq_length])
        init_state = tf.placeholder(tf.float32,[None, state_size])

        x_one_hot = tf.one_hot(x,vocab_size)
        y_one_hot = tf.one_hot(y,vocab_size)

        rnn_inputs = tf.unstack(x_one_hot,axis=1)
        rnn_labels = tf.unstack(y_one_hot,axis=1)

        state = init_state
        rnn_outputs = []
        for rnn_input in rnn_inputs:
            state = rnn_cell(rnn_input, state)
            rnn_outputs.append(state)
        final_state = state

        with tf.variable_scope(str(id(self)) + 'softmax'):
            w = tf.get_variable('w',[state_size, vocab_size])
            b = tf.get_variable('b',[vocab_size])

        logits = [tf.matmul(rnn_output, w) + b for rnn_output in rnn_outputs]
        #predictions = [tf.nn.softmax(logit) for logit in logits]

        losses = [tf.nn.softmax_cross_entropy_with_logits_v2(labels=label, logits=logit) \
                  for logit, label in zip(logits, rnn_labels)]
        total_loss = tf.reduce_mean(losses)
        update = tf.train.AdagradOptimizer(learning_rate).minimize(total_loss)
        
        return x,y,init_state,final_state,total_loss,update
    
    def train(self,num_epochs):
        x,y,init_state,final_state,total_loss,update = self.create_compute_graph()
        
        self.sess.run(tf.global_variables_initializer())
        training_losses=[]
        e_index = 0
        for epoch in get_epoch(num_epochs):
            
            training_loss = 0
            training_state = np.zeros((batch_size, state_size))
            for step, (X, Y) in enumerate(epoch):
                training_loss_, training_state, _ = self.sess.run([total_loss,final_state,update],
                                                             feed_dict={x:X, y:Y, init_state:training_state})
                training_loss += training_loss_
            if e_index % 10 == 0 and e_index > 0:
                print("Average loss at epoch", e_index,
                      "for last 10 epochs:", training_loss/100)
                training_losses.append(training_loss/100)
                training_loss = 0
            e_index+=1
        print('train finished')
        return training_losses
    
    def create_test_graph(self):
        x = tf.placeholder(tf.int32,[1])
        x_one_hot = tf.one_hot(x,vocab_size)
        init_state = tf.placeholder(tf.float32,[1,state_size])
        
        with tf.variable_scope(str(id(self)) + 'rnn_cell',reuse=True):
            w = tf.get_variable('w',[vocab_size + state_size, state_size])
            b = tf.get_variable('b',[state_size])
            
        state = tf.tanh(tf.matmul(tf.concat([x_one_hot,init_state],axis=1),w) + b)
        
        with tf.variable_scope(str(id(self)) + 'softmax', reuse=True):
            w2 = tf.get_variable('w',[state_size, vocab_size])
            b2 = tf.get_variable('b',[vocab_size])
        y = tf.matmul(state,w2) + b2
        p = tf.nn.softmax(y)
        out = tf.argmax(p,axis=1) 
        return x, init_state,state, out
    
    def sample(self,n):
        x, init_state, state,out = self.create_test_graph()
        test_x = np.array([char2idx.get(data[0])])
        training_state = np.zeros([1,state_size])
        result = []
        for i in range(n):
            result.append(test_x[0])
            training_state,test_x = self.sess.run([state,out],feed_dict = {x:test_x, 
                                                                           init_state:training_state})
        return "".join(list(map(lambda x:idx2char[x],result)))
        


In [6]:
model = CharRnnModel()   
print('start traning...')

model.train(100)
print('start testing...')
model.sample(100)

start traning...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Average loss at epoch 10 for last 10 epochs: 0.7381861710548401
Average loss at epoch 20 for last 10 epochs: 0.5140705299377442
Average loss at epoch 30 for last 10 epochs: 0.3861145704984665
Average loss at epoch 40 for last 10 epochs: 0.3100062629580498
Average loss at epoch 50 for last 10 epochs: 0.246302972137928
Average loss at epoch 60 for last 10 epochs: 0.2039945475757122
Average loss at epoch 70 for last 10 epochs: 0.17008764773607254
Average loss at epoch 80 for last 10 epochs: 0.1492837519943714
Average loss at epoch 90 for last 10 epochs: 0.13327353641390802
train finished
start testing...


'# lodid             rnite_date)\n    brrat_camistep,in srate, bo sel(btes)\n\n                         '